# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in the City CSV File

city_file = os.path.join('..',"WeatherPy","City_Data.csv")
file = open(city_file)
csvreader = csv.DictReader(city_file)
  
city_data_df = pd.read_csv(city_file,index_col=0)
city_data_df

  
 
                      


city country  latitude  longitude  humidity  cloudiness  \
0            russkiy      RU   43.0147   131.8642      67.0        74.0   
1    manavalakurichi      IN    8.1333    77.3000      74.0         3.0   
2             dikson      RU   73.5069    80.5464      97.0        98.0   
3     saint-philippe      RE  -21.3585    55.7679      94.0       100.0   
4      saint anthony      US   45.0205   -93.2180      73.0         0.0   
..               ...     ...       ...        ...       ...         ...   
619        kalangala      UG   -0.3089    32.2250      67.0        95.0   
620            lafia      NG    8.4833     8.5167       8.0         3.0   
621           cuamba      MZ  -14.8031    36.5372      71.0        42.0   
623            kyren      RU   51.6828   102.1408      89.0        88.0   
624         wheeling      US   42.1392   -87.9290      87.0       100.0   

     windspeed  temperature  
0        10.92         1.26  
1         4.99        84.29  
2        14.16        -4.72  
3        29.51        79.59  
4         5.01        -2.25  
..         ...          ...  
619      12.84        74.07  
620       1.21        98.22  
621       2.19        81.88  
623       1.95         2.97  
624      11.50        18.10  

[578 rows x 8 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#  Configure gmaps 
gmaps.configure(api_key=g_key)

In [34]:
#  Using Lat/Long and Humidity as the Weight, 
locations = city_data_df[["latitude", "longitude"]]
humidity = city_data_df["humidity"]

heat_map = gmaps.figure(zoom_level= 2,center = (0,0))
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, opacity=.5,dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 3


heat_map.add_layer(heat_layer)
heat_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_df = city_data_df[(city_data_df['temperature'] >= 75) & (city_data_df['temperature'] <= 85)]
perfect_df = perfect_df[perfect_df['windspeed'] <= 10]
perfect_df = perfect_df[perfect_df['cloudiness'] <= 10]
perfect_df = perfect_df[perfect_df['humidity'] <= 60]
hotel_df = perfect_df
hotel_df

city country  latitude  longitude  humidity  cloudiness  \
160        tanout      NE   14.9709     8.8879       6.0         3.0   
164          chuy      UY  -33.6971   -53.4616      50.0         0.0   
195   san alberto      CO    7.7611   -73.3922      53.0         0.0   
240  buenos aires      AR  -34.6132   -58.3772      50.0         1.0   
307       salalah      OM   17.0151    54.0924      39.0         0.0   
437          bani      DO   18.2833   -70.3333      59.0         1.0   
593        bhalki      IN   18.0333    77.2167      19.0         2.0   

     windspeed  temperature  
160       8.68        78.93  
164       6.76        79.93  
195       4.00        82.51  
240       8.81        84.67  
307       5.75        80.69  
437       9.31        79.23  
593       4.41        78.64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Add a column for the Hotel Name 
hotel_df['Hotel Name'] = ""
hotel_df

city country  latitude  longitude  humidity  cloudiness  \
160        tanout      NE   14.9709     8.8879       6.0         3.0   
164          chuy      UY  -33.6971   -53.4616      50.0         0.0   
195   san alberto      CO    7.7611   -73.3922      53.0         0.0   
240  buenos aires      AR  -34.6132   -58.3772      50.0         1.0   
307       salalah      OM   17.0151    54.0924      39.0         0.0   
437          bani      DO   18.2833   -70.3333      59.0         1.0   
593        bhalki      IN   18.0333    77.2167      19.0         2.0   

     windspeed  temperature Hotel Name  
160       8.68        78.93             
164       6.76        79.93             
195       4.00        82.51             
240       8.81        84.67             
307       5.75        80.69             
437       9.31        79.23             
593       4.41        78.64

In [21]:
# Iterate through the Hotel DataFrame and find Hotels within 5000 meters for the Lat/Long

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
 
        params['location'] = f"{latitude}, {longitude}"
        latitude = row['latitude']
        longitude = row['longitude']

        
        hotel_info = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "None Found" # If no hotel is found, put "None Found"
        
hotel_df        

city country  latitude  longitude  humidity  cloudiness  \
160        tanout      NE   14.9709     8.8879       6.0         3.0   
164          chuy      UY  -33.6971   -53.4616      50.0         0.0   
195   san alberto      CO    7.7611   -73.3922      53.0         0.0   
240  buenos aires      AR  -34.6132   -58.3772      50.0         1.0   
307       salalah      OM   17.0151    54.0924      39.0         0.0   
437          bani      DO   18.2833   -70.3333      59.0         1.0   
593        bhalki      IN   18.0333    77.2167      19.0         2.0   

     windspeed  temperature               Hotel Name  
160       8.68        78.93   Mohammed biryani hotel  
164       6.76        79.93               None Found  
195       4.00        82.51       Turis Firper Hotel  
240       8.81        84.67  Hotel San Alberto Plaza  
307       5.75        80.69      Hilton Buenos Aires  
437       9.31        79.23    Salalah Gardens Hotel  
593       4.41        78.64            Dpravia Hotel

In [22]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heat_map.add_layer(markers)



# Display figure
heat_map

Figure(layout=FigureLayout(height='420px'))